In [1]:
import pandas as pd

In [2]:
quality = pd.read_csv("manufacturing_data_JunAug2025.csv")
oee = pd.read_csv("production_oee_data_JunAug2025.csv")
inventory = pd.read_csv("inventory_data_JunAug2025.csv")

In [3]:
print(quality.head())

         Date  Shift  Machine_No Product_ID  Operator  Qty_Produced  Defects  \
0  01-06-2025      1           1       53mm  A.Shaikh         54899      338   
1  01-06-2025      1           2       74mm  L.Jadhav         67045      198   
2  01-06-2025      1           3       53mm  L.Jadhav         72660      205   
3  01-06-2025      1           4       82mm  B.Sawale        195349      512   
4  01-06-2025      1           5       82mm  A.Shaikh        133819     1295   

   Breakdown_Min  
0              0  
1             15  
2              5  
3             15  
4             20  


In [4]:
print(oee.head())

         Date  Shift  Machine_No Product  Operator  Planned_Time_Min  \
0  01-06-2025      1           1    82mm  A.Shaikh               480   
1  01-06-2025      1           2    53mm   P.Singh               480   
2  01-06-2025      1           3    74mm   M.Kadam               480   
3  01-06-2025      1           4    53mm  B.Sawale               480   
4  01-06-2025      1           5    82mm  B.Sawale               480   

   Run_Time_Min  Downtime_Min  Qty_Produced  Scrap  Standard_Rate  
0           420            60         12750    193           1900  
1           465            15         16544    412           2200  
2           420            60         15504    768           2100  
3           465            15         18532    879           2200  
4           450            30         15561    662           1900  


In [5]:
print(inventory.head())

         Date Product Warehouse          Supplier  Opening_Stock  Inward  \
0  01-06-2025    35mm    Nashik         Beta Corp           1250       0   
1  01-06-2025    38mm      Pune  Delta Industries            962       0   
2  01-06-2025    53mm    Nashik     Gamma Traders           1008       0   
3  01-06-2025    74mm      Pune     Gamma Traders           1976       0   
4  01-06-2025    82mm      Pune  Delta Industries           1488       0   

   Outward  Closing_Stock  Lead_Time_Days  
0        0           1250               6  
1      384            578               3  
2        0           1008               4  
3        0           1976               9  
4        0           1488               8  


In [6]:
quality.columns = quality.columns.str.lower().str.replace(" ", "_")
oee.columns = oee.columns.str.lower().str.replace(" ", "_")
inventory.columns = inventory.columns.str.lower().str.replace(" ", "_")

In [7]:
# Calculate defect percentage
quality["defect_pct"] = (quality["defects"] / quality["qty_produced"]) * 100

In [8]:
# Check average defects by machine
quality_summary = quality.groupby("machine_no")["defect_pct"].mean().reset_index()

In [9]:
print("Average defect % by machine:")
print(quality_summary)

Average defect % by machine:
   machine_no  defect_pct
0           1    0.560343
1           2    0.579995
2           3    0.561962
3           4    0.535007
4           5    0.548983
5           6    0.553088


In [10]:
quality.to_csv("clean_quality_defects.csv", index=False)

In [11]:
# Availability = Run Time / Planned Time
oee["availability"] = oee["run_time_min"] / oee["planned_time_min"]

In [12]:
# Performance = (Standard Rate * Qty Produced) / Run Time
oee["performance"] = (oee["standard_rate"] * oee["qty_produced"]) / oee["run_time_min"]

In [13]:
# Quality = (Qty Produced - Scrap) / Qty Produced
oee["quality"] = (oee["qty_produced"] - oee["scrap"]) / oee["qty_produced"]

In [14]:
# OEE = Availability * Performance * Quality
oee["oee"] = oee["availability"] * oee["performance"] * oee["quality"]

In [15]:
# Average OEE by machine
oee_summary = oee.groupby("machine_no")["oee"].mean().reset_index()

In [16]:
print("Average OEE by machine:")
print(oee_summary)

Average OEE by machine:
   machine_no           oee
0           1  60722.650589
1           2  58859.860734
2           3  58717.076540
3           4  58670.764266
4           5  59311.081295
5           6  59968.879076


In [17]:
oee.to_csv("clean_oee.csv", index=False)

In [18]:
# Calculate total stock value
inventory["total_value"] = inventory["closing_stock"] * 100

In [19]:
# Sort by highest stock value
inventory_sorted = inventory.sort_values("total_value", ascending=False)

In [20]:
print("Top inventory items by value:")
print(inventory_sorted.head(10))

Top inventory items by value:
           date product warehouse          supplier  opening_stock  inward  \
215  14-07-2025    35mm    Mumbai         Alpha Ltd           1979     394   
231  17-07-2025    38mm    Mumbai     Gamma Traders           1969     373   
299  30-07-2025    82mm      Pune         Beta Corp           1930     389   
59   12-06-2025    82mm    Mumbai         Alpha Ltd           1914     394   
282  27-07-2025    53mm    Mumbai         Alpha Ltd           1947     492   
171  05-07-2025    38mm    Nashik         Alpha Ltd           1942     265   
271  25-07-2025    38mm    Mumbai  Delta Industries           1973     191   
99   20-06-2025    82mm    Nashik     Gamma Traders           1937     215   
8    02-06-2025    74mm      Pune         Alpha Ltd           1931     215   
47   10-06-2025    53mm    Mumbai  Delta Industries           1893     242   

     outward  closing_stock  lead_time_days  total_value  
215        0           2373               8       23

In [21]:
inventory.to_csv("clean_inventory.csv", index=False)

In [22]:
summary = pd.merge(
    quality_summary, oee_summary,
    left_on="machine_no", right_on="machine_no", how="inner"
)

In [23]:
summary.rename(columns={"defect_pct": "avg_defect_pct", "oee": "avg_oee"}, inplace=True)

In [24]:
print("Combined Machine Summary:")
print(summary)

Combined Machine Summary:
   machine_no  avg_defect_pct       avg_oee
0           1        0.560343  60722.650589
1           2        0.579995  58859.860734
2           3        0.561962  58717.076540
3           4        0.535007  58670.764266
4           5        0.548983  59311.081295
5           6        0.553088  59968.879076


In [25]:
summary.to_csv("machine_summary.csv", index=False)